In [1]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
mini_batch=128

In [3]:
train_data=MNIST('./datas/mnist',train=True,download=False,transform=transform)
test_data=MNIST('./datas/mnist',train=False,download=False,transform=transform)

In [4]:
train_loader=DataLoader(dataset=train_data,batch_size=mini_batch,shuffle=True,num_workers=8)
test_loader=DataLoader(dataset=test_data,batch_size=mini_batch,shuffle=False,num_workers=8)

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1=torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2=torch.nn.Conv2d(10,20,kernel_size=5)
        self.pooling=torch.nn.MaxPool2d(kernel_size=2)
        self.linear1=torch.nn.Linear(320,256)
        self.linear2=torch.nn.Linear(256,128)
        self.linear3=torch.nn.Linear(128,64)
        self.linear4=torch.nn.Linear(64,10)
    def forward(self,x):
        batch_size=x.size(0)
        x=self.pooling(F.relu(self.conv1(x)))
        x=self.pooling(F.relu(self.conv2(x)))
        x=x.view(batch_size,-1)
        x=F.relu(self.linear1(x))
        x=F.relu(self.linear2(x))
        x=F.relu(self.linear3(x))
        x=self.linear4(x)
        return x
model=Model()

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Model(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=320, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=64, bias=True)
  (linear4): Linear(in_features=64, out_features=10, bias=True)
)

In [7]:
criterion=torch.nn.CrossEntropyLoss(size_average=True)
optimizer=torch.optim.SGD(params=model.parameters(),lr=0.01,momentum=0.5)

c:\users\root\appdata\local\programs\python\python37\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [1]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader):
        x,y=data
        x,y=x.to(device),y.to(device)
        optimizer.zero_grad()
        y_=model(x)
        loss=criterion(y_,y)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if batch_idx%500==0:
            print('[%d, %5d] loss: %.3f'%(epoch+1,batch_idx+1,running_loss/500))
            running_loss=0.0
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            x,y=data
            x,y=x.to(device),y.to(device)
            y_=model(x)
            _,predicted=torch.max(y_.data,dim=1)
            total+=y.size(0)
            correct+=(predicted==y).sum().item()
    print('Accuracy=',100*correct/total)

In [9]:
%%time
for epoch in range(10):
    train(epoch)
    test()

[1,     1] loss: 0.005
Accuracy= 89.07
[2,     1] loss: 0.001
Accuracy= 94.36
[3,     1] loss: 0.000
Accuracy= 96.33
[4,     1] loss: 0.000
Accuracy= 97.12
[5,     1] loss: 0.000
Accuracy= 97.36
[6,     1] loss: 0.000
Accuracy= 97.82
[7,     1] loss: 0.000
Accuracy= 98.13
[8,     1] loss: 0.000
Accuracy= 98.22
[9,     1] loss: 0.000
Accuracy= 98.18
[10,     1] loss: 0.000
Accuracy= 98.43
Wall time: 1min 16s
